In [ ]:
#https://python.langchain.com/docs/integrations/vectorstores/pgembedding

# PostgreSQL as Vector Database with LangChain

In this lab you will utilize the built in Document Loading capabilities of LangChain to embed documents into PostgreSQL.

You will then use a similarity search to search the indexed and embedded documents followed by using a query Retriever to add context to an Open AI request.

In [ ]:
!pip install langchain
!pip install langchain-openai
!pip install psycopg2
!pip install tiktoken

In [ ]:
import os
import psycopg2
from pgvector.psycopg2 import register_vector

from typing import List, Tuple
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter

#depening on your langchain version...
#from langchain_community.document_loaders import TextLoader
#from langchain_community.document_loaders import PdfLoader
#from langchain_community.vectorstores import PGEmbedding
#from langchain_openai import OpenAIEmbeddings

from langchain.document_loaders import UnstructuredXMLLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.pgvector import PGVector
from langchain.vectorstores.pgvector import DistanceStrategy

username = 'wsuser'
password = 'Solliance123'
host = 'pgsqldevilkdittaflex16.postgres.database.azure.com'
port = '5432'
dbname = 'ailabs'

connection_string = f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{dbname}'

In [ ]:
dbconn = psycopg2.connect(host=host, user=username, password=password,
    port=port, database=dbname , connect_timeout=10)
dbconn.set_session(autocommit=True)

cur = dbconn.cursor()
cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")

register_vector(dbconn)

In [ ]:
#get the openai embeddings
embedding_model = "embeddings" #this is the name of the model deployment in azure open ai (not the type of model)
azure_endpoint = "YOUR_API_URL"  #https://fllm4693d-openai.openai.azure.com/
azure_key = 'YOUR_API_KEY'  #abcdf0a8efa1432796a3dab8f0e61234

azure_endpoint = "https://fllm4693d-openai.openai.azure.com/"
azure_key = '1b13f0a8efa1432796a3dab8f0e62a2d'

embeddings = OpenAIEmbeddings(
                deployment=embedding_model,
                openai_api_base=azure_endpoint,
                openai_api_key=azure_key,
                openai_api_type="azure",
            )

In [ ]:
import logging
logFormatter = logging.Formatter("%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s")
rootLogger = logging.getLogger()

logPath = "./logs"
fileName = "langchain"

fileHandler = logging.FileHandler("{0}/{1}.log".format(logPath, fileName))
fileHandler.setFormatter(logFormatter)
rootLogger.addHandler(fileHandler)

consoleHandler = logging.StreamHandler()
consoleHandler.setFormatter(logFormatter)
rootLogger.addHandler(consoleHandler)

rootLogger.setLevel(logging.DEBUG)

In [ ]:
def insert_document(db, doc, collection_name=None, collection_id=None):

    import uuid

    #generate the embeddings
    doc_embeddings = embeddings.embed_documents(doc.page_content)

    #collection = db.get_collection(db._make_session())

    collection_id = "a1a2bb38-562b-4d46-9475-268fe3bc4e4d"

    #insert with manual sql
    sql = "INSERT INTO langchain_pg_embedding (collection_id, embedding, document, cmetadata, custom_id, uuid) VALUES ({0},ARRAY {1}, {2}, {3},{4}, {5})".format(collection_id, doc_embeddings, doc.page_content, None, None, uuid.uuid4())
    dbconn.execute_sql(sql)

In [ ]:
loader = PyPDFLoader("./data/azure_openai_infographic.pdf")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

collection_name = "open_ai_pdf"

db = PGVector(
    collection_name=collection_name,
    connection_string=connection_string,
    embedding_function=embeddings,
)

In [ ]:
db = db.from_documents(
    documents= docs,
    embedding = embeddings,
    collection_name= collection_name,
    distance_strategy = DistanceStrategy.COSINE,
    connection_string=connection_string,
    logger=rootLogger
)

In [ ]:
for doc in docs:
    insert_document(db, doc, collection_name=collection_name)

In [ ]:
#utilize a simple similarity search
query = "What did the president say about Ketanji Brown Jackson"

docs_with_score: List[Tuple[Document, float]] = db.similarity_search_with_score(query)

for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

In [ ]:
#use the text loader and splitter to break apart the document into chunks
loader = TextLoader("./data/msft_earnings_call_transcript.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

collection_name = "msft_earnings"

db = PGVector(
    collection_name=collection_name,
    connection_string=connection_string,
    embedding_function=embeddings,
)

#you can do a full database load with the creation of the collection and the embedding table.
#db = db.from_documents(
#    documents= docs,
#    embedding = embeddings,
#    collection_name= collection_name,
#    distance_strategy = DistanceStrategy.COSINE,
#    connection_string=connection_string)

#you can also manually import the documents into a target collection
for i in range(0, len(docs), 1):
    temp_docs = docs[i:i+1]
    db.add_documents( documents=temp_docs, collection_name=collection_name, connection_string=connection_string)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import ConversationalRetrievalChain

In [ ]:
#utilize a query retriver
prompt_prefix = "You are a question and answering system. You are given a question and a context. You must answer the question based on the context provided. {context} Question: {question}"
deployment_name = "completions"
api_version = ""
model_version = ""

question = "Which analytis were on the call?"

retriever = db.as_retriever()

#create a simple QA chain
prompt = PromptTemplate(
    template=prompt_prefix,
    input_variables=["context", "question"], #"summaries", "question"
)

llm = AzureChatOpenAI(deployment_name=deployment_name,
                        temperature=0,
                        openai_api_base=azure_endpoint,
                        openai_api_key=azure_key,
                        openai_api_type="azure",
                        openai_api_version=api_version,
                        model_version=model_version)

llm_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    return_source_documents=False,
    #memory=memory,
    #callbacks = [self.handler],
    chain_type="stuff",
    combine_docs_chain_kwargs={"prompt": prompt},
    verbose=True
)

answer = llm_chain.invoke(question, return_only_outputs=True)['answer']

print(answer)
